In [2]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
from google.colab import files
import io

# Upload the file
uploaded = files.upload()

# Read the content of the uploaded .xlsx file
# Use io.BytesIO instead of io.StringIO because .xlsx is a binary file
with io.BytesIO(uploaded['TeamHealthRawDataForDemo.xlsx']) as f:
    df = pd.read_excel(f, engine='openpyxl')

df["row_id"] = df.index + 1
# Display the content of the DataFrame
print(df.head(10))

Saving TeamHealthRawDataForDemo.xlsx to TeamHealthRawDataForDemo.xlsx
    Period Manager    Team                                           Response  \
0  2019-Q1   Mgr 1  Team 1  We're a fun team that works well together and ...   
1  2019-Q1   Mgr 1  Team 1  we have a sound and collaborative team focused...   
2  2019-Q1   Mgr 1  Team 1  we work well as a team, we have fun together, ...   
3  2019-Q1   Mgr 1  Team 1  I fell pretty good about the health of our tea...   
4  2019-Q1   Mgr 1  Team 1  happy with team's overall health and good dyna...   
5  2019-Q1   Mgr 1  Team 1                                              Solid   
6  2019-Q1   Mgr 1  Team 1  The Team 2 team is a  collaborative group prod...   
7  2019-Q1   Mgr 1  Team 1  We have great teamwork.  We have a lot of fun....   
8  2019-Q1   Mgr 1  Team 1  We feel good about our teamwork, process, tech...   
9  2019-Q1   Mgr 1  Team 2  A blast!  Always working towards delivering mo...   

   row_id  
0       1  
1       2  
2 

In [8]:
#create a new data frame with "id" and "comment" fields
df_subset = df[['row_id', 'Response']].copy()
#data clean-up
#remove all non-aphabet characters
df_subset['Response'] = df_subset['Response'].str.replace("[^a-zA-Z#]", " ")
#covert to lower-case
df_subset['Response'] = df_subset['Response'].str.casefold()
print (df_subset.head(10))

   row_id                                           Response
0       1  we're a fun team that works well together and ...
1       2  we have a sound and collaborative team focused...
2       3  we work well as a team, we have fun together, ...
3       4  i fell pretty good about the health of our tea...
4       5  happy with team's overall health and good dyna...
5       6                                              solid
6       7  the team 2 team is a  collaborative group prod...
7       8  we have great teamwork.  we have a lot of fun....
8       9  we feel good about our teamwork, process, tech...
9      10  a blast!  always working towards delivering mo...


In [25]:
# set up empty dataframe for staging output
df1=pd.DataFrame()
df1['row_id']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

**Sentiment Type** values are
- **neg** for negative sentiment
- **neu** for neutral sentiment
- **pos** for positive sentiment
- **compound** for an overall score that combines negative, positive, and neutral sentiments into a single score.



In [26]:
print('Processing sentiment analysis...')
# Initialize the sentiment analyzer
sid = SentimentIntensityAnalyzer()

t_df = df1

# Iterate over each row in df_subset
#iterrows() เป็นเมธอดของ DataFrame ในไลบรารี pandas ที่ใช้สำหรับการวนลูปผ่านแถวของ DataFrame โดยให้ข้อมูลเป็นคู่ของดัชนี (index) และแถว (row)
for index, row in df_subset.iterrows():
    # Access the text using .iloc to avoid deprecation warning
    #row.iloc เป็นวิธีการเข้าถึงข้อมูลใน pandas DataFrame/Series โดยใช้ index position แทนการใช้ชื่อคอลัมน์หรือแถว
    text = row.iloc[1]  # Assuming the text is in the second column
    scores = sid.polarity_scores(text)  # Returns 'neg', 'neu', 'pos', and 'compound'

    # Iterate through the sentiment scores and their types
    for key, value in scores.items():
        row_id = row.iloc[0]  # Assuming the row_id is in the first column
        df1['row_id']=row_id
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=pd.concat([t_df,df1])

# Remove dummy rows with row_id = '99999999999' if any
t_df_cleaned = t_df[t_df['row_id'] != '99999999999']

# Remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()

# Only keep rows where sentiment_type = 'compound'
t_df_cleaned = t_df_cleaned[t_df_cleaned['sentiment_type'] == 'compound']

# Display the first 10 rows of the cleaned DataFrame
print(t_df_cleaned.head(10))

Processing sentiment analysis...
   row_id sentiment_type  sentiment_score
0       1       compound           0.6597
0       2       compound           0.9287
0       3       compound           0.8122
0       4       compound           0.8225
0       5       compound           0.8271
0       6       compound           0.1531
0       7       compound           0.9382
0       8       compound           0.9381
0       9       compound           0.9468
0      10       compound           0.5519


In [27]:
#merge dataframes
df_output = pd.merge(df, t_df_cleaned, on='row_id', how='inner')
print(df_output.head(10))

    Period Manager    Team                                           Response  \
0  2019-Q1   Mgr 1  Team 1  We're a fun team that works well together and ...   
1  2019-Q1   Mgr 1  Team 1  we have a sound and collaborative team focused...   
2  2019-Q1   Mgr 1  Team 1  we work well as a team, we have fun together, ...   
3  2019-Q1   Mgr 1  Team 1  I fell pretty good about the health of our tea...   
4  2019-Q1   Mgr 1  Team 1  happy with team's overall health and good dyna...   
5  2019-Q1   Mgr 1  Team 1                                              Solid   
6  2019-Q1   Mgr 1  Team 1  The Team 2 team is a  collaborative group prod...   
7  2019-Q1   Mgr 1  Team 1  We have great teamwork.  We have a lot of fun....   
8  2019-Q1   Mgr 1  Team 1  We feel good about our teamwork, process, tech...   
9  2019-Q1   Mgr 1  Team 2  A blast!  Always working towards delivering mo...   

   row_id sentiment_type  sentiment_score  
0       1       compound           0.6597  
1       2       comp